<a href="https://colab.research.google.com/github/MohibShaikh/Twitter-Sentiment-Analysis/blob/main/Sentiment_Analysis_(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohibus","key":"c3af835593e5a5c51a78897e3e4154d0"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets list

ref                                                         title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                         Online Food Dataset                              3KB  2024-03-02 18:50:30          19229        388  0.9411765        
sahirmaharajj/electric-vehicle-population-size-2024         Electric Vehicle Population by Country (2024)  275KB  2024-03-30 19:16:06           1112         32  1.0              
mexwell/drug-consumption-classification                     💊 Drug Consumption Classification               56KB  2024-03-28 15:02:22           1592         34  1.0              
bhavikjikadara/student-study-performance                    Student Study Performance                    

In [6]:
!kaggle datasets download -d krishbaisoya/tweets-sentiment-analysis

 99% 39.0M/39.3M [00:01<00:00, 40.5MB/s]
100% 39.3M/39.3M [00:01<00:00, 30.6MB/s]


In [7]:
from zipfile import ZipFile
path = '/content/tweets-sentiment-analysis.zip'
with ZipFile(path,'r') as zip:
  zip.extractall()
  print('Dataset Extracted!')

Dataset Extracted!


In [14]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
print(stopwords.words('german'))

['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anderr', 'anders', 'auch', 'auf', 'aus', 'bei', 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass', 'daß', 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe', 'dazu', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich', 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein', 'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger', 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es', 'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer', 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier', 'hin', 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euc

### Data Preprocessing

In [28]:
# Loading Data
twt_data = pd.read_csv('/content/train_data.csv', encoding='ISO-8859-1')

In [29]:
twt_data.head()

,sentence,sentiment
0,awww that s a bummer you shoulda got david car...,0
1,is upset that he can t update his facebook by ...,0
2,i dived many times for the ball managed to sav...,0
3,my whole body feels itchy and like its on fire,0
4,no it s not behaving at all i m mad why am i h...,0


In [30]:
twt_data.shape

(1523975, 2)

In [27]:
twt_data.describe()

,sentiment
count,1.523975e+06
mean,4.966722e-01
std,4.999891e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [31]:
twt_data.tail()

,sentence,sentiment
1523970,just woke up having no school is the best feel...,1
1523971,thewdb com very cool to hear old walt interviews,1
1523972,are you ready for your mojo makeover ask me fo...,1
1523973,happy th birthday to my boo of alll time tupac...,1
1523974,happy charitytuesday,1


In [34]:
twt_data.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [36]:
twt_data['sentiment'].value_counts()

sentiment
0    767059
1    756916
Name: count, dtype: int64

In [38]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]','',content)